# **Setup**

In [1]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

Cloning into 'SemEval_2022_Task2-idiomaticity'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 123 (delta 48), reused 61 (delta 15), pack-reused 0
Receiving objects: 100% (123/123), 2.50 MiB | 11.00 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

Cloning into 'AStitchInLanguageModels'...
remote: Enumerating objects: 1030, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 1030 (delta 11), reused 4 (delta 4), pack-reused 1013
Receiving objects: 100% (1030/1030), 79.59 MiB | 20.43 MiB/s, done.
Resolving deltas: 100% (394/394), done.


In [3]:
!pip install transformers
%cd /content/ 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.6 MB/s eta 0:00:00
/content


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
import site
site.main()

In [6]:
import os
import csv

from pathlib import Path

# **Pre-process Data**

In [7]:
def load_csv( path, delimiter=',' ) : 
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile, delimiter=delimiter ) 
    for row in reader : 
      if header is None : 
        header = row
        continue
      data.append( row ) 
  return header, data

In [8]:
def write_csv( data, location ) : 
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile ) 
    writer.writerows( data ) 
  print( "Wrote {}".format( location ) ) 
  return

In [9]:
class Node():
  def __init__(self, sentence, label):
    self.sentence = sentence
    self.label = label

def create_idiom_dict_train(data_location, file_name) :
    idiom_dict = {}
    file_name = os.path.join( data_location, file_name ) 
    header, data = load_csv( file_name )
    for elem in data:
        label     = elem[ header.index( 'Label'  ) ]
        sentence = elem[ header.index( 'Target' ) ]
        idiom = elem[ header.index( 'MWE' ) ]
        if idiom in idiom_dict:
          idiom_dict[idiom].append(Node(sentence, label))
        else:
          idiom_dict[idiom] = [Node(sentence, label)]
    return idiom_dict

In [10]:
d1 = create_idiom_dict_train('SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', 'train_zero_shot.csv')
d2 = create_idiom_dict_train('SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', 'train_one_shot.csv')
for key, value in d2.items():
  if key in d1:
    d1[key].append(value)
  else:
    d1[key] = value

In [11]:
def _get_train_data( data_location, file_name, include_context, include_idiom ) :
    
    file_name = os.path.join( data_location, file_name ) 

    header, data = load_csv( file_name )

    out_header = [ 'label1', 'label2', 'sentence1', 'sentence3' ]
    if include_idiom :
        out_header = [ 'label1', 'label2', 'sentence1', 'sentence2', 'sentence3', 'sentence4' ]
        
    # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
    out_data = list()
    for elem1 in data :
        label     = elem1[ header.index( 'Label'  ) ]
        sentence1 = elem1[ header.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem1[ header.index( 'Previous' ) ], elem1[ header.index( 'Target' ) ], elem1[ header.index( 'Next' ) ] ] )
        for elem2 in d1[elem1[ header.index( 'MWE' ) ]]:
          if elem2.sentence != sentence1:
              label2     = elem2.label
              sentence2 = elem2.sentence
              this_row = None
              if not include_idiom :
                  this_row = [ label, label2, sentence1, sentence2 ] 
              else :
                  sentence3 = elem1[ header.index( 'MWE' ) ]
                  sentence4 = sentence3
                  this_row = [ label, label2, sentence1, sentence3, sentence2, sentence4]
              out_data.append( this_row )
              assert len( out_header ) == len( this_row )
    return [ out_header ] + out_data

In [12]:
def _get_dev_eval_data( data_location, input_file_name, gold_file_name, include_context, include_idiom ) :

    input_headers, input_data = load_csv( os.path.join( data_location, input_file_name ) )
    gold_header  = gold_data = None
    if not gold_file_name is None : 
        gold_header  , gold_data  = load_csv( os.path.join( data_location, gold_file_name  ) )
        assert len( input_data ) == len( gold_data )

    # ['ID', 'Language', 'MWE', 'Previous', 'Target', 'Next']
    # ['ID', 'DataID', 'Language', 'Label']
    
    out_header = [ 'label1', 'label2', 'sentence1', 'sentence3' ]
    if include_idiom :
        out_header = [ 'label1', 'label2', 'label3', 'sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5', 'sentence6', 'language' ]

    out_data = list()
    for index in range( len( input_data ) ) :
        label = 1
        if not gold_file_name is None : 
            this_input_id = input_data[ index ][ input_headers.index( 'ID' ) ]
            this_gold_id  = gold_data [ index ][ gold_header  .index( 'ID' ) ]
            assert this_input_id == this_gold_id
            
            label     = gold_data[ index ][ gold_header.index( 'Label'  ) ]
            language = gold_data[index][gold_header.index('Language')]
        elem      = input_data[ index ]
        sentence1 = elem[ input_headers.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ input_headers.index( 'Previous' ) ], elem[ input_headers.index( 'Target' ) ], elem[ input_headers.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ input_headers.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        idiom = elem[ input_headers.index( 'MWE' ) ]
        other_nodes = d1[idiom]
        if(len(other_nodes)==1):
            if not include_idiom :
                this_row = [ label, other_nodes[0].label, sentence1, other_nodes[0].sentence ] 
            else :
                sentence2 = elem[ input_headers.index( 'MWE' ) ]
                this_row = [ label, other_nodes[0].label, other_nodes[0].label, sentence1, sentence2, other_nodes[0].sentence, sentence2, other_nodes[0].sentence, sentence2, language ]
        else:
            if not include_idiom :
                this_row = [ label, other_nodes[0].label, sentence1, other_nodes[0].sentence ] 
            else :
                sentence2 = elem[ input_headers.index( 'MWE' ) ]
                this_row = [ label, other_nodes[0].label, other_nodes[1].label, sentence1, sentence2, other_nodes[0].sentence, sentence2, other_nodes[1].sentence, sentence2, language ]
           
        assert len( out_header ) == len( this_row ) 
        out_data.append( this_row )
        

    return [ out_header ] + out_data

In [13]:
def create_data( input_location, output_location ) :

    
    ## Zero shot data
    train_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = True,
        include_idiom   = False
    )
    write_csv( train_data, os.path.join( output_location, 'ZeroShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = True,
        include_idiom    = False
    )        
    write_csv( dev_data, os.path.join( output_location, 'ZeroShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    write_csv( eval_data, os.path.join( output_location, 'ZeroShot', 'eval.csv' ) )


    ## OneShot Data (combine both for training)
    train_zero_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = False,
        include_idiom   = True
    )
    train_one_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_one_shot.csv',
        include_context = False,
        include_idiom   = True
    )

    assert train_zero_data[0] == train_one_data[0] ## Headers
    train_data = train_one_data + train_zero_data[1:]
    write_csv( train_data, os.path.join( output_location, 'OneShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = False,
        include_idiom    = True
    )        
    write_csv( dev_data, os.path.join( output_location, 'OneShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None,
        include_context  = False,
        include_idiom    = True
    )
    write_csv( eval_data, os.path.join( output_location, 'OneShot', 'eval.csv' ) )

    return

In [14]:
train_zero_data = _get_train_data(
        data_location   = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
        file_name       = 'train_zero_shot.csv',
        include_context = False,
        include_idiom   = True
    )
train_one_data = _get_train_data(
        data_location   = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
        file_name       = 'train_one_shot.csv',
        include_context = False,
        include_idiom   = True
    )

assert train_zero_data[0] == train_one_data[0] ## Headers
train_data = train_one_data + train_zero_data[1:]

dev_data = _get_dev_eval_data(
        data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = False,
        include_idiom    = True
    )

In [15]:
!ls

AStitchInLanguageModels  sample_data  SemEval_2022_Task2-idiomaticity


In [16]:
outpath = 'Data'

Path( os.path.join( outpath, 'ZeroShot' ) ).mkdir(parents=True, exist_ok=True)
Path( os.path.join( outpath, 'OneShot' ) ).mkdir(parents=True, exist_ok=True)

# create_data( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', outpath )

# **Implementation of Siamese Network**

In [17]:
import os
import sys
import numpy as np
import random
import pickle
import logging

from typing          import Optional
from dataclasses     import dataclass, field
from sklearn.metrics import f1_score, accuracy_score

from datasets        import load_dataset, load_metric

import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.utils         import check_min_version
from transformers.trainer_utils import get_last_checkpoint, is_main_process

In [18]:
# Will error if the minimal version of Transformers is not installed. Remove at your own risks.
check_min_version("4.6.0.dev0")

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

logger = logging.getLogger(__name__)

In [19]:
train_data[0:4]

[['label1', 'label2', 'sentence1', 'sentence2', 'sentence3', 'sentence4'],
 ['0',
  '1',
  'Program leaders said the scholarship defines public service broadly and imagines a variety of pathways toward civic engagement.',
  'public service',
  'In the ensuing years, Wennberg might not have managed to knock down the parking deck, but his administration helped keep Central Vermont Public Service from moving its corporate headquarters out of the city, and after successfully fighting a number of shopping centers city officials worried would pose a threat to downtown, he negotiated a deal that kept Diamond Run Mall from hosting a movie theater or supermarket and got the city a couple million dollars in payments that funded a variety of projects through the years.',
  'public service'],
 ['1',
  '0',
  'In the ensuing years, Wennberg might not have managed to knock down the parking deck, but his administration helped keep Central Vermont Public Service from moving its corporate headquarters 

In [20]:
dev_data[0:4]

[['label1',
  'label2',
  'label3',
  'sentence1',
  'sentence2',
  'sentence3',
  'sentence4',
  'sentence5',
  'sentence6',
  'language'],
 ['1',
  '1',
  '1',
  'Are these interruptions of the good life a necessary condition of the high life?',
  'high life',
  'Despite having the riches to afford the high life, PSG captain Marquinhos is still in touch with his past life before becoming a multi-millionaire footballer.',
  'high life',
  'Despite having the riches to afford the high life, PSG captain Marquinhos is still in touch with his past life before becoming a multi-millionaire footballer.',
  'high life',
  'EN'],
 ['1',
  '1',
  '1',
  "But for Australian fashion designer Abby Kheir, there's no reason not to treat her employees to a taste of the high life all-year round.",
  'high life',
  'Despite having the riches to afford the high life, PSG captain Marquinhos is still in touch with his past life before becoming a multi-millionaire footballer.',
  'high life',
  'Despite ha

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from keras_preprocessing.sequence import pad_sequences
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large',
                                          cache_dir=None,
                                          use_fast=True,
                                          revision="main",
                                          use_auth_token=None,)

Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [22]:
def shuffle_data(data):
    indices = list(range(len(data)))
    random.shuffle(indices)
    shuffled_data = []
    for i in indices:
        shuffled_data.append(data[i])
    return shuffled_data

# **Tokenizing**

In [23]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

def Preprocess_Data(input, tokenizer, max_len, batch_size, data_class="train"):
    input1 = []
    input2 = []
    label1 = []
    label2 = []
    for i in input:
      """if(i[1]!='1' and i[1]!='0'):
        continue"""
      label1.append(int(i[0]))
      label2.append(int(i[1]))
      args = (
            (i[2], i[3])
      )
      input1.append(args)
      args = (
            (i[4], i[5])
      )
      input2.append(args)
    encoded_input1 = tokenizer(input1, padding='max_length', max_length = max_len, truncation=True, return_tensors="pt")
    encoded_input2 = tokenizer(input2, padding='max_length', max_length = max_len, truncation=True, return_tensors="pt")
    
    input_ids1 = encoded_input1['input_ids']
    attention_mask1 = encoded_input1['attention_mask']
    labels1 = torch.tensor(label1)

    #print(input_ids1.size(), attention_mask1.size(), labels1.size())

    input_ids2 = encoded_input2['input_ids']
    attention_mask2 = encoded_input2['attention_mask']
    labels2 = torch.tensor(label2)

    #print(input_ids2.size(), attention_mask2.size(), labels2.size())
    dataset_tensor = TensorDataset(input_ids1, attention_mask1, labels1, input_ids2, attention_mask2, labels2)

    if data_class == "train":
        sampler = RandomSampler(dataset_tensor)
    else:
        sampler = SequentialSampler(dataset_tensor)
    dataloader = DataLoader(dataset_tensor, sampler=sampler, batch_size=batch_size)

    return dataloader
#train_dataloader = preproces(train_data[1:], tokenizer, max_len, batch_size, data_class="train")
#dev_dataloader = preproces(eval_data[1:], tokenizer, max_len, batch_size, data_class="dev")
#test_dataloader = preproces(dev_Data[1:], tokenizer, max_len, batch_size, data_class="test")

In [24]:
max_len = 512
batch_size = 32
def PreProcess_Dev(input, tokenizer, max_len, batch_size, data_class="dev"):
    input1 = []
    input2 = []
    input3 = []
    label1 = []
    label2 = []
    label3 = []
    language = []
    for i in input:
      """if(i[1]!='1' and i[1]!='0'):
        continue"""
      label1.append(int(i[0]))
      label2.append(int(i[1]))
      label3.append(int(i[2]))
      args = (
            (i[3], i[4])
      )
      input1.append(args)
      args = (
            (i[5], i[6])
      )
      input2.append(args)
      args = (
          (i[7], i[8])
      )
      input3.append(args)
      if i[9]=='EN':
        language.append(0)
      else:
        language.append(1)
    encoded_input1 = tokenizer(input1, padding='max_length', max_length = max_len, truncation=True, return_tensors="pt")
    encoded_input2 = tokenizer(input2, padding='max_length', max_length = max_len, truncation=True, return_tensors="pt")
    encoded_input3 = tokenizer(input3, padding='max_length', max_length = max_len, truncation=True, return_tensors="pt")
    input_ids1 = encoded_input1['input_ids']
    attention_mask1 = encoded_input1['attention_mask']
    labels1 = torch.tensor(label1)
    print(input_ids1.size(), attention_mask1.size(), labels1.size())

    input_ids2 = encoded_input2['input_ids']
    attention_mask2 = encoded_input2['attention_mask']
    labels2 = torch.tensor(label2)

    print(input_ids2.size(), attention_mask2.size(), labels2.size())

    input_ids3 = encoded_input3['input_ids']
    attention_mask3 = encoded_input3['attention_mask']
    labels3 = torch.tensor(label3)

    print(input_ids3.size(), attention_mask3.size(), labels3.size())  
    language = torch.tensor(language)  

    dataset_tensor = TensorDataset(input_ids1, attention_mask1, labels1, input_ids2, attention_mask2, labels2, input_ids3, attention_mask3, labels3, language)

    if data_class == "train":
        sampler = RandomSampler(dataset_tensor)
    else:
        sampler = SequentialSampler(dataset_tensor)
    dataloader = DataLoader(dataset_tensor, sampler=sampler, batch_size=batch_size)

    return dataloader
#dev_dataloader = preproces_dev(dev_data[1:], tokenizer, max_len, batch_size, data_class="dev")

# **Siamese Model**

In [25]:
import torch.nn as nn
class SiameseModel(nn.Module):
    def __init__(self):
        super(SiameseModel, self).__init__()
        
        self.base_model = AutoModel.from_pretrained(
            'xlm-roberta-large',
            from_tf=bool(".ckpt" in 'xlm-roberta-large'),
            config=config,
            cache_dir=None,
            revision="main",
            use_auth_token=None,
        ).cuda()
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(768, 2).cuda() 
        
    def forward(self, input_ids1, attn_mask1, input_ids2, attn_mask2):
        
        outputs1 = self.base_model(input_ids1, attention_mask=attn_mask1).last_hidden_state[:, 0]
        outputs2 = self.base_model(input_ids2, attention_mask=attn_mask2).last_hidden_state[:, 0]
        difference = outputs1*outputs2
        
        outputs = self.dropout(difference)
        outputs = self.linear(outputs)
        
        return outputs

In [26]:
from sklearn.metrics import f1_score, accuracy_score
def Eval(bert_model, dataloader):

    bert_model.eval()
    predictions, true_labels = [], []
    predictions_en, true_labels_en = [], []
    predictions_pt, true_labels_pt = [], []
    num_correct = 0
    
    for step, batch in enumerate(tqdm(dataloader)):
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            logits1 = nn.functional.softmax(bert_model.forward(batch[0], batch[1], batch[3], batch[4]), -1)
        with torch.no_grad():
            logits2 = nn.functional.softmax(bert_model.forward(batch[0], batch[1], batch[6], batch[7]), -1)
        logits = torch.cat((logits1, logits2), dim=1)
        max_args = torch.argmax(logits, dim=1)
        batch_predictions = []
        batch_true_labels = batch[2]
        first_sentence_labels = batch[5]
        second_sentence_labels = batch[8]
        batch_en_predictions = []
        batch_pt_predictions = []
        true_en_predictions = []
        true_pt_predictions = []
        language = batch[9]
        for idx, instance in enumerate(max_args):
          if instance == 0:
            batch_predictions.append((first_sentence_labels[idx] - 1) * -1) # 0, 1 toggle
            if language[idx].item() == 0:
              batch_en_predictions.append((first_sentence_labels[idx] - 1) * -1)
              true_en_predictions.append(batch_true_labels[idx])
            else:
              batch_pt_predictions.append((first_sentence_labels[idx] - 1) * -1)
              true_pt_predictions.append(batch_true_labels[idx])
          elif instance == 1:
            batch_predictions.append(first_sentence_labels[idx])
            if language[idx].item() == 0:
              batch_en_predictions.append(first_sentence_labels[idx])
              true_en_predictions.append(batch_true_labels[idx])
            else:
              batch_pt_predictions.append(first_sentence_labels[idx])
              true_pt_predictions.append(batch_true_labels[idx])
          elif instance == 2:
            batch_predictions.append((second_sentence_labels[idx] - 1) * -1)
            if language[idx].item() == 0:
              batch_en_predictions.append((second_sentence_labels[idx] - 1) * -1)
              true_en_predictions.append(batch_true_labels[idx])
            else:
              batch_pt_predictions.append((second_sentence_labels[idx] - 1) * -1)
              true_pt_predictions.append(batch_true_labels[idx])
          else:
            batch_predictions.append(second_sentence_labels[idx])
            if language[idx].item() == 0:
              batch_en_predictions.append(second_sentence_labels[idx])
              true_en_predictions.append(batch_true_labels[idx])
            else:
              batch_pt_predictions.append(second_sentence_labels[idx])
              true_pt_predictions.append(batch_true_labels[idx])
        predictions += batch_predictions
        true_labels += batch_true_labels
        predictions_en += batch_en_predictions
        predictions_pt += batch_pt_predictions
        true_labels_en += true_en_predictions
        true_labels_pt += true_pt_predictions
    return true_labels, predictions, true_labels_en, predictions_en, true_labels_pt, predictions_pt

In [27]:
def metrics(true_labels, predictions):
    pre = []
    tl = []
    num_correct = 0
    for pred, true_label in zip(predictions, true_labels):
        pre.append(int(pred.item()))
        tl.append(int(true_label.item()))
        if pred == true_label:
            num_correct += 1
    print("\nAccuracy: %s" % (float(num_correct) / float(len(true_labels))))
    print("F1 Score ")
    print(f1_score(tl, pre, average='macro'))

In [28]:
from tqdm import tqdm

def Train_Eval(bert_model, train_data, lr, n_epoch, tokenizer, batch_size, max_len):

    print("Start Training!")
    optimizer = AdamW(bert_model.parameters(), lr=lr)
    bert_model.train()
    shuffled_train_data = shuffle_data(train_data)
    #tokenizing training data
    shuffled_train_data = Preprocess_Data(shuffled_train_data, tokenizer, max_len, batch_size)
    #tokenizing testing data
    dev_dataloader = PreProcess_Dev(dev_data[1:], tokenizer, max_len, batch_size, data_class="dev")
    # TRAIN loop
    for epoch in range(n_epoch):
        print(f"\nEpoch {epoch}")
        torch.cuda.empty_cache()
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(shuffled_train_data)):
            batch = tuple(t.to(device) for t in batch)
            bert_model.zero_grad()
            # forward pass
            logits = bert_model.forward(batch[0], batch[1], batch[3], batch[4])

            loss = 0
            target = torch.where(batch[2]==batch[5], 1, 0)
            
            loss = nn.functional.cross_entropy(logits, target)
            
            # backward pass
            loss.backward()
            # track train loss
            tr_loss += loss.item()
            nb_tr_steps += 1
            #loss = loss.detach()
            # update parameters
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            

        # print train loss per epoch
        print("Train loss on epoch {}: {}\n".format(epoch, tr_loss / nb_tr_steps))

    true_labels, predictions, true_labels_en, predictions_en, true_labels_pt, predictions_pt  = Eval(bert_model, dev_dataloader)
    print("EN-PT")
    metrics(true_labels, predictions)
    print("EN")
    metrics(true_labels_en, predictions_en)
    print("PT")
    metrics(true_labels_pt, predictions_pt)


In [29]:
config = AutoConfig.from_pretrained(
        'xlm-roberta-large',
        num_labels=2,
        finetuning_task=None,
        cache_dir=None,
        revision="main",
        use_auth_token=None,
    )

# **Training and Testing the Model**

In [30]:
model = SiameseModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

learning_rate = 2e-5
num_epoch = 3                          
torch.cuda.empty_cache()
max_len = 128
batch_size = 64

if n_gpu > 1:
    model.to(device)
    model = torch.nn.DataParallel(model)
else:
    model.cuda()

Train_Eval(model, train_data[1:], learning_rate, num_epoch, tokenizer, batch_size, max_len)


Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start Training!


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


torch.Size([739, 128]) torch.Size([739, 128]) torch.Size([739])
torch.Size([739, 128]) torch.Size([739, 128]) torch.Size([739])
torch.Size([739, 128]) torch.Size([739, 128]) torch.Size([739])

Epoch 0


  0%|          | 0/1347 [00:02<?, ?it/s]


RuntimeError: ignored

In [ ]:
# %%shell
# jupyter nbconvert --to html 1_XLM_Roberta_Siamese .ipynb